In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
line_color = '#f1f5f9'
font_color = '#f1f5f9'
background_color = '#032137'

layout_dict = dict(
    font_color=font_color,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis = dict(gridcolor=line_color, linecolor=line_color, zerolinecolor=line_color),
    yaxis = dict(gridcolor=line_color, linecolor=line_color, zerolinecolor=line_color),
    legend=dict(x=0, y=1, bgcolor=background_color),
    margin=dict(r=0, l=0, b=0)
)

tablet_args = dict(default_width='700px', default_height='300px')
mobile_args = dict(default_width='400px', default_height='175px')

In [4]:
def bound_array(a, bounds=[0, 1]):
    s = a.copy()

    # Check number of times it breaks bounds
    signed_crosses = np.floor((s - bounds[0]) / (bounds[1] - bounds[0])).astype(int)
    crosses = np.floor(np.where(signed_crosses < 0, -signed_crosses, signed_crosses)).astype(int)

    # Reorient direction
    s *= ((-1) ** crosses)

    # Reposition to maintain continuity
    s += (bounds[1] - bounds[0]) * signed_crosses * (-1) ** (crosses + 1)
    s += (bounds[1] + bounds[0]) * 0 ** ((crosses + 1) % 2)

    return s

def non_stationary_bounded_random_walk(initial=1/2, bounds=[0, 1], size=[1000, 1], drift=0, std=0.001, std_bounds=[0, 0.05]):
    # Generate stds
    std_array = abs(np.random.normal(loc=drift, scale=std, size=size).cumsum(0) + std)
    std_array = bound_array(std_array, std_bounds)   

    # Generate random walk initialized on 0
    walk = np.random.normal(loc=drift, scale=std_array, size=size).cumsum(0) + initial
    walk = bound_array(walk, bounds)        

    return walk

# Nice example plot
walks = non_stationary_bounded_random_walk(size=1000)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(y=walks, opacity=0.5, name='Random Walk'), secondary_y=False)
# fig.add_trace(go.Scatter(y=std_array, opacity=0.5, name='Std'), secondary_y=True)

fig.update_layout(**layout_dict, title='Non Stationary Random Walk')
fig.update_yaxes(range=[-0.2, 1.2], secondary_y=False)
fig.update_yaxes(range=[-0.01, 0.06], secondary_y=True)
fig.show()

# Approximating Standard Deviation

In [9]:
def apply_rolling_function(array, window, func):
    shape = list(array.shape)
    res = np.zeros(shape)

    for i in range(shape[0]):
        res[i] = func(array[max(0, i - window):min(shape[0], i + window)])

    return res

def step_mean(array, alpha, init=0.5):
    N = array.shape
    means = alpha * np.ones(N) * init + (1 - alpha) * array[0]

    for i in range(1, N[0]):
        means[i] = alpha * means[i - 1] + (1 - alpha) * array[i]

    return means


In [73]:
N = 1000
p, std = non_stationary_bounded_random_walk(size=N)
games_results = (np.random.random(N) <= p).astype(int)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(y=p, opacity=0.5, name='Mean'), secondary_y=False)
fig.add_trace(go.Scatter(y=std, opacity=0.5, name='Std'), secondary_y=True)

fig.update_layout(**layout_dict, title='Non Stationary Random Walk')
fig.update_yaxes(range=[-0.2, 1.2], secondary_y=False)
fig.update_yaxes(range=[-0.01, 0.06], secondary_y=True)
fig.show()

In [75]:
fig = go.Figure(layout=layout_dict | dict(title='Comparison of Mean Approximations'))

fig.add_trace(go.Scatter(y=p, line=dict(color='white'), name='Actual Mean'))

windows = [5, 10, 20, 50, 100]
alphas = [0.85, 0.9, 0.95, 0.975, 0.99]
for i in range(len(windows)):
    mu_window = apply_rolling_function(games_results, windows[i], np.mean)
    step_mu = step_mean(games_results, alphas[i])

    fig.add_trace(go.Scatter(y=mu_window, name=f'Mean window = {windows[i]}', opacity=0.7))
    fig.add_trace(go.Scatter(y=step_mu, name=f'Mean alpha = {alphas[i]}', opacity=0.7))

fig.show()

In [80]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(y=std ** 2, name='Actual std'), secondary_y=False)

windows = [5, 10, 20, 50, 100]
alpha = 0.8
for i in range(1, len(windows)):
    std_window = apply_rolling_function(games_results, windows[i], np.std)
    std_step = step_mean(games_results, 0.9)
    std_step = apply_rolling_function(std_step, windows[i], np.std)

    fig.add_trace(go.Scatter(y=std_window ** 2, name=f'std window = {windows[i]}', opacity=0.7), secondary_y=True)
    fig.add_trace(go.Scatter(y=std_step ** 2, name=f'Step std window = {windows[i]}', opacity=0.7), secondary_y=True)

fig.update_layout(**layout_dict, title='Comparison of Standar Deviation Approximations')
# fig.update_yaxes(range=[-0.00005, 0.003], secondary_y=False)
# fig.update_yaxes(range=[-0.05, 0.3], secondary_y=True)
fig.show()


In [64]:
N = 1000
p, std = non_stationary_bounded_random_walk(size=N)
games_results = (np.random.random(N) <= p).astype(int)


fig = go.Figure(layout=layout_dict | dict(title='Comparison of Mean Approximations'))

fig.add_trace(go.Scatter(y=p, line=dict(color='white'), name='Actual Mean'))

windows = [5, 10, 20, 50, 100]
alphas = [0.1, 0.25, 0.5, 0.75, 0.9]
for i in range(len(windows)):
    mu_window = apply_rolling_function(games_results, windows[i], np.mean)
    step_mu = step_mean(games_results, alphas[i])

    fig.add_trace(go.Scatter(y=mu_window, name=f'Mean window = {windows[i]}', opacity=0.7))
    fig.add_trace(go.Scatter(y=step_mu, name=f'Mean alpha = {alphas[i]}', opacity=0.7))

fig.show()

## TO DO:

Create a generator to train several models with:

- $\boldsymbol{Y}$: 
    - Non-stationary Bounded Random Walk (with random inits, drifts and stds)
- $\boldsymbol{X}$:
    - n: which iteration is it (first iterations have more uncertainty).
    - window means
    - alpha means
    - window stds
    - window alpha stds

In [47]:
def generate_X(games_results: np.array, windows: list = [5, 10, 20, 50, 100], alphas: list = [0.85, 0.9, 0.95, 0.975, 0.99]):
    # Generate indexes
    shape = games_results.shape
    random_walk_ids = np.array([[i] * shape[1] for i in range(shape[0])]).flatten().tolist()

    # Calculate window means, stds
    window_means = np.empty([len(windows), 0]).tolist()
    window_stds = np.empty([len(windows), 0]).tolist()
    for i in range(len(windows)):
        window_means[i] = apply_rolling_function(games_results, windows[i], np.mean).flatten().tolist()
        window_stds[i] = apply_rolling_function(games_results, windows[i], np.std).flatten().tolist()

    # Calculate step means, stds
    step_means = np.empty([len(alphas), 0]).tolist()
    step_stds = np.empty([len(alphas), 0]).tolist()
    for i in range(len(alphas)):
        step_means[i] = step_mean(games_results, alphas[i])
        step_stds[i] = apply_rolling_function(step_means[i], 10, np.std).flatten().tolist()

    X = np.array(
        random_walk_ids +
        np.array(games_results).flatten().tolist() +
        np.array(window_means).flatten().tolist() +
        np.array(window_stds).flatten().tolist() +
        np.array(step_means).flatten().tolist() +
        np.array(step_stds).flatten().tolist()
    ).reshape([2 + 2 * len(windows) + 2 * len(alphas), -1])

    return X



def data_generator(n_par: int, n_runs: int, depth: int, windows: list = [5, 10, 20, 50, 100], alphas: list = [0.85, 0.9, 0.95, 0.975, 0.99]):
    # Controlling variables
    N = [depth, n_runs]
    params = zip(np.random.rand(n_par), np.random.rand(n_par) * 0.01, np.random.rand(n_par) * 0.1)
    params = [
        {
            'initial': item[0],
            'drift': item[1],
            'std': item[2]
        } for item in params
    ]

    y = []
    X = np.empty([2 + 2 * len(windows) + 2 * len(alphas), 0]).tolist()
    for param in params:
        # Generate Y
        p = non_stationary_bounded_random_walk(**param, size=N)
        y += p.flatten().tolist()

        # Calculate games results
        games_results = (np.random.random(N) <= p).astype(int)

        # Fill X
        _X = generate_X(games_results, windows=windows, alphas=alphas).tolist()
        for i in range(len(X)):
            X[i] += _X[i]

    X = np.array(X)
    y = np.array(y)
    return X, y

data_generator(2, 3, 5)

(array([[0.        , 0.        , 0.        , 1.        , 1.        ,
         1.        , 2.        , 2.        , 2.        , 3.        ,
         3.        , 3.        , 4.        , 4.        , 4.        ,
         0.        , 0.        , 0.        , 1.        , 1.        ,
         1.        , 2.        , 2.        , 2.        , 3.        ,
         3.        , 3.        , 4.        , 4.        , 4.        ],
        [0.        , 0.        , 0.        , 0.        , 1.        ,
         0.        , 0.        , 0.        , 1.        , 1.        ,
         1.        , 0.        , 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
         0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
         0.33333333, 0.33333333,